In [5]:
import requests
import datetime
import base64

In [6]:
#From Spotify API
client_id = 'd22e1351aa4a4ddfb7c0550137efdd7a'
client_secret = '7f23c73eb6cb447ea7dbb1840b43d778'

In [7]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    token_headers = None

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs) 
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """ 
        Returns base 66 encoded string 
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception('Must set client id and client secret')
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds_b64}'
        }

    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data = token_data, headers = token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [8]:
spotify = SpotifyAPI(client_id, client_secret)

In [9]:
spotify.perform_auth()

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

In [10]:
access_token = spotify.access_token
access_token